# 3. Logging

## 3.1 Logging Basics

### 3.1.1 로그란?

- 데이터는 이제 우리의 삶 어디에서나 존재
- 앱을 사용할 때마다 우리가 어떤 행동을 하는지 데이터가 남음
- 이런 데이터를 사용자 로그 데이터, 이벤트 로그 데이터 등으로 부름
- 위처럼 **머신러닝 인퍼런스 요청 로그, 인퍼런스 결과**를 저장해야 함

<br>

### 3.1.2 데이터의 종류

#### 3.1.2.1 데이터베이스 데이터 (서비스 로그, Database에 저장)

- 서비스가 운영되기 위해 필요한 데이터
- ex) 고객이 언제 가입했는 지, 어떤 물건을 구입했는 지 등

<br>

#### 3.1.2.2 사용자 행동 데이터 (유저 행동 로그, 주로 Object Storage, 데이터 웨어하우스에 저장)

- 유저 로그라고 지칭하면 사용자 행동 데이터를 의미
- 서비스에 반드시 필요한 내용은 아니고, 더 좋은 제품을 만들기 위해 또는 데이터 분석시 필요한 데이터
- 앱이나 웹에서 유저가 어떤 행동을 하는지를 나타내는 데이터
- UX와 관련해서 인터랙션이 이루어지는 관점에서 발생하는 데이터
  - ex) Click, View, 스와이프 등

<br>

#### 3.1.2.3 인프라 데이터 (Metric)

- 백엔드 웹 서버가 제대로 동작하고 있는 지 확인하는 데이터
- Request 수, Response 수
- DB 부하 수

<br>

#### 3.1.2.4 Metric, Log, Trace

Metric

- 값을 측정할 때 사용
- CPU, Memory 등

Log

- 운영 관점에서 알아야 하는 데이터를 남길 때 사용
- 함수가 호출되었다. 예외 처리가 되었다 등

Trace

- 개발 관점에서 알아야하는 것
- 예외 Trace

<br>

### 3.1.3 저장된 데이터 활용 방식

1. "image.jpg" 로 마스크 분류 모델로 요청했다
  - "image.jpg"를 중간에 Object Storage에 저장하면 실제로 우리가 볼 때의 실제 Label과 예측 Label을 파악할 수 있음
2. "image.jpg" 같은 이름의 이미지로 10번 요청했다
  - 같은 이미지로 예측한다고 하면 중간에 저장해서 기존에 예측한 결과를 바로 Return할 수 있겠다.
  - Redis 등을 사용해 캐싱
3. `Feature = [[2, 5, 10, 4]]` 으로 수요 예측 모델로 요청했다
  - 어떤 Feature가 들어오는 지 알 수 있고, Feature를 사용할 때 모델이 어떻게 예측하는 지 알 수 있음
4. 현재 시스템이 잘 동작하는 지 알 수 있음

<br>

데이터가 저장되어 있지 않다면

- 과거에 어떤 예측을 했는지 알 수 없음
- print 문의 로그를 저장한 곳을 찾아서 확인해야 함(Linux 서버에 접속하거나)
- 모델이 더 발전하기 위한 개선점을 찾기 어려움
- 현재 시스템이 잘 동작하고 있는지 알기 어려움

<br>

### 3.1.4 데이터 적재 방식

1. Database(RDB)에 저장하는 방식
  - 다시 웹, 앱 서비스에서 사용되는 경우 활용
  - 실제 서비스용 Database
2. Database(NoSQL)에 저장하는 방식
  - Elasticsearch, Logstash or Fluent, Kibana에서 활용하려는 경우
3. Object Storage에 저장하는 방식
  - S3, Cloud Storage에 파일 형태로 저장
  - csv, parquet, json 등
  - 별도로 Database나 Data Warehouse로 옮기는 작업이 필요
4. Data Warehouse에 저장하는 방식
  - 데이터 분석시 활용하는 데이터 웨어하우스로 바로 저장

<br>

#### 3.1.4.1 RDBMS

- 관계형 데이터베이스(Relational Database)
- 행과 열로 구성
- 데이터의 관계를 정의하고, 데이터 모델링 진행
- 비즈니스와 연관된 중요한 정보
  - ex) 고객 정보, 주문 요청
- 영구적으로 저장해야 하는 것은 데이터베이스에 저장
- 데이터 추출시 SQL 사용
- MySQL, PostgreSQL 등

<br>

#### 3.1.4.2 NoSQL

- 스키마가 Strict한 RDBMS와 다르게 스키마가 없거나 느슨한 스키마만 적용
- Not Only SQL
- 데이터가 많아지며 RDBMS로 트래픽을 감당하기 어려워서 개발됨
- 일반적으로 RDBMS에 비해 쓰기와 읽기 성능이 빠름
- Key Value Store, Document, Column Family, Graph 등
- JSON 형태와 비슷하며 XML 등도 활용됨 
```
{
    key : value,
    key2 : {
        key2-1:value21,
        key2-2:value22
    }
}
```
- MongoDB

<br>

#### cf) SQL vs NoSQL

<img src="https://k21academy.com/wp-content/uploads/2020/12/asesoftware-sql-nosql.png" width=500 />

<br>

#### 3.1.4.3 Object Storage

- 어떤 형태의 파일이여도 저장할 수 있는 저장소
- AWS S3, GCP Cloud Storage 등
- 특정 시스템에서 발생하는 로그를 xxx.log에 저장한 후, Object Storage에 저장하는 형태
- 비즈니스에서 사용되지 않는 분석을 위한 데이터
- 이미지, 음성 등을 저장

<br>

#### 3.1.4.4 Data Warehouse

- 여러 공간에 저장된 데이터를 한 곳으로 저장
- 데이터 창고
- RDBMS, NoSQL, Object Storage 등에 저장된 데이터를 한 곳으로 옮겨서 처리
- RDBMS와 같은 SQL을 사용하지만 성능이 더 좋은 편
- AWS Redshift, GCP BigQuery, Snowflake 등

<br>

#### 3.1.4.5 실습 시 데이터 적재 방식

- 정말 다양한 방법으로 데이터를 저장할 수 있음  
- 상황에 따라 다르지만, 여기선 Serving 과정에서 데이터를 기록하기 위함  
- 상황
  - 데이터 분석은 BigQuery에서 진행
  - 해당 로그는 애플리케이션과 상관없음
- 사용 가능 대안
  - 파이썬 로깅 모듈을 사용해 CSV로 저장해서 Cloud Storage에 업로드
  - BigQuery로 바로 데이터 추가

<br>

## 3.2 Logging in Python

### 3.2.1 Python Logging Module

#### 3.2.1.1 `logging`

- 파이썬 기본 모듈
- 웹서버, 머신러닝, CLI 등 여러 파이썬 코드에서 사용할 수 있음
- 심각도(Severity)에 따라 info, debug, error, warning 등 다양한 카테고리로 데이터를 저장할 수 있음

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zGfIx7xRgKKaJTYT1awsAGhSOZBLeM8l' width=400/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zGjmtPgCXm0Mj_wHpkiMZV_x-nu77gej' width=800/>

<br>

#### 3.2.1.2 Log Level

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zHEsHTrGoTl8FGV9HISDlT2BG6csQp1x' width=800/>

- 기본 Logging Level은 WARNING
- 설정하지 않으면 WARNING보다 심각한 레벨(WARNING, ERROR, CRITICAL)의 로그만 보여줌

<br>

#### 3.2.1.3 `logging` vs `print`

- console에만 output을 출력하는 print
- logging은 file, websocket 등 파이썬이 다룰 수 있는 모든 포맷으로 output을 출력할 수 있음
  - **언제 어디서**(파일 이름과 코드 상의 몇번째 줄인지) 해당 output이 발생했는지 알 수 있음
  - output을 **심각도**에 따라 분류할 수 있음
  - ex) Dev 환경에서는 debug 로그까지, Prod(운영) 환경에서는 info 로그만 보기 등)
  - 다만 print보다 알아야 하는 지식이 존재

<br>

#### 3.2.1.4 `logging_module.py`

- `examples/logging_module.py` 참고
- `logging` 모듈에서 제공하는 정보(ex. asctime, levelname 등)는 아래 링크에서 확인할 수 있음
  - [https://docs.python.org/3/library/logging.html#logrecord-attributes](https://docs.python.org/3/library/logging.html#logrecord-attributes)

<br>

### 3.2.2 Python Logging Component

- Loggers
- Handlers
- Filters
- Formatters

- `examples/dynamic_logger.py` 참고

<br>

#### 3.2.2.1 Logger

- 로그를 생성하는 Method 제공
  - `logger.info()` 등
- 로그 Level과 Logger에 적용된 Filter를 기반으로 처리해야 하는 로그인지 판단
- Handler에게 LogRecord 인스턴스 전달

`logging.getLogger(name)`으로 Logger Object 사용

- name이 주어지면 해당 name의 logger 사용하고, name이 없으면 root logger 사용
- 마침표로 구분되는 계층 구조
  - `logging.getLogger('foo.bar')` => `logging.getLogger('foo')`의 자식 logger 반환

`logging.setLevel()`
  - Logger에서 사용할 Level 지정

<br>

#### 3.2.2.2 Handler

- Logger에서 만들어진 Log를 적절한 위치로 전송(파일 저장 또는 Console 출력 등)
- Level과 Formatter를 각각 설정해서 필터링 할 수 있음
- StreamHandler, FileHandler, HTTPHandler 등

<img src="https://uploads.toptal.io/blog/image/125745/toptal-blog-image-1521816113545-53ceb5c3ff0270a6a38de42009a185ea.png" width=800 />

<br>

#### 3.2.2.3 Formatter

- 최종적으로 Log에 출력될 Formatting 설정
- 시간, Logger 이름, 심각도, Output, 함수 이름, Line 정보, 메시지 등 다양한 정보 제공

<img src="https://uploads.toptal.io/blog/image/125744/toptal-blog-image-1521816083917-1ea6e4e1f49467fefc169917c34b0d1f.png" width=800 />

<br>

### 3.2.3 Logging Flow

<img src="https://docs.python.org/3.7/_images/logging_flow.png" width=800 />

<br>

## 3.3 Online Serving Logging (BigQuery)

**BigQuery에 Online Serving Input과 Output 로그 적재**

1. 빅쿼리 테이블을 세팅합니다.
2. 빅쿼리에 적재하기 쉽게 JSON 형태로 로그를 적재
  - [pythonjsonlogger](https://github.com/madzak/python-json-logger)를 사용
  - JSON 형태로 저장하는 Logger: `examples/json_logging.py`
3. python logging 모듈을 사용해서 빅쿼리에(실시간) 로그 적재
  - file과 console에도 남을 수 있도록 handler를 지정

<br>

**BigQuery**

- Google Cloud Platform의 데이터 웨어하우스
- 데이터 분석을 위한 도구로 Apache Spark의 대용으로 활용 가능
- Firebase, Google Analytics 4와 연동되어 많은 회사에서 사용 중

<br>

### 3.3.1 BigQuery 데이터 구조

- GCP의 Project 내부에 BigQuery의 리소스가 존재
  - Dataset, Table, View 등

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zHYLrTw3Ar8s6iogdJug50W22sikGdT1' width=600/>

<br>

### 3.3.2 BigQuery 데이터세트 만들기

- GCP Console에서 BigQuery 접근 - API 사용 - 데이터 세트 만들기 클릭
- 적절한 이름의 데이터세트 생성

<br>

### 3.3.3 BigQuery 테이블 만들기

- 방금 생성한 online_serving_logs 데이터셋에서 Create Table 클릭
- 파티션 설정
  - 빅쿼리는 데이터를 조회할 때 모든 데이터를 조회하지 않고 일부를 조회하기 때문에 비용을 줄이기 위한 방법

<br>

### 3.3.4 서비스 계정 생성

- 빅쿼리 데이터 로깅을 위해 서비스 계정 생성
- 새 키 만들기 - JSON Key 다운

<br>

### 3.3.5 BigQuery로 실시간 로그 데이터 수집하기

- `credential_json_path`에 위에서 발급받은 서비스 계정 JSON 파일의 경로를 입력하면 python sdk에서 해당 파일을 읽고 권한을 부여받을 수 있음

```
credentials = service_account.Credentials.from_service_account_file(
    filename=credential_json_path
)
```

- 빅쿼리 client를 만들어 query를 실행하고, 테이블 정보를 불러올 수 있음

```
self.config = config
self.bigquery_client = bigquery.Client(credentials=self.config.credentials)
self.setLevel(config.level)
```

<br>

`app` 폴더 내의 파일들 참고

<br>

#### 3.3.5.1 `config.yaml` 파일

formatter에 대한 구성 설정

- simple
  - 일반 포맷팅
- json
  - json으로 로그를 변환
  - `()`: logger config에서 사용할 클래스를 지정
- pythonjsonlogger의 `JsonFormatter` 클래스를 사용하면 일반 텍스트를 json으로 변환

```yaml
version: 1
formatters:
  simple:
    format: '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
  json:
    (): pythonjsonlogger.jsonlogger.JsonFormatter
    format: '%(asctime)s, %(filename)s, %(levelname)s, %(name)s, %(message)s'
    json_encoder: json.JSONEncoder
    rename_fields:
      message: 'log_message'
```

<br>

handlers와 loggers, root logger를 정의

- console handler는 기본 StreamHandler를 사용해서 콘솔/터미널에 로그를 출력
- logfile handler는 파일에 로그 저장
- maxbytes, backupCount 등도 설정 가능

```yaml
handlers:
  console:
    class: logging.StreamHandler
    level: INFO
    formatter: json
    stream: ext://sys.stdout
  logfile:
    class: logging.handlers.RotatingFileHandler
    level: INFO
    formatter: json
    filename: ./app/log/info.log
    maxBytes: 10485760 # 10MB
    backupCount: 20
loggers:
  sampleLogger:
    level: INFO
    handlers: [console,logfile]
    propagate: no
root:
  level: INFO
  handlers: [console,logfile]
```

<br>

#### 3.3.5.2 `logger.py` - `get_ml_logger()`

- 위에서 정의한 logger config를 기반으로 로그를 생성하는 함수 정의

logger에 config 주입

```python
with open(config_path, "r") as f:
    logging_config = yaml.safe_load(f) # 설정은 config.yaml을 load
logging.config.dictConfig(logging_config) # logging.config에 주입
_logger = logging.getLogger(logger_name) # 해당 config가 적용된 logger를 만듬
```

<br>

빅쿼리 핸들러 추가

- 빅쿼리 핸들러에는 `BigqueryHandlerConfig`라는 클래스로 관련 설정들을 만들고, 주입
- 윗 부분에서 만든 `logger`에 `BigqueryHandler`를 추가

```python
# BigQuery Logging Handler 추가합니다
if not credential_json_path:
    return _logger

credentials = service_account.Credentials.from_service_account_file(
    filename=credential_json_path
)
bigquery_handler_config = BigqueryHandlerConfig(
    credentials=credentials,
    table=table_ref,
    formatter=jsonlogger.JsonFormatter(fmt=log_format),
)
bigquery_handler = BigqueryHandler(config=bigquery_handler_config)
_logger.addHandler(bigquery_handler)

return _logger
```

<br>

#### 3.3.5.3 `logger.py` - `BigQueryHandlerConfig` 클래스

- Pydantic 커스텀 클래스를 사용하기 위해 `arbitrary_types_allowed = True`

```python
class BigqueryHandlerConfig(BaseSettings):
    credentials: service_account.Credentials
    table: Union[str, bigquery.TableReference]
    formatter: logging.Formatter = Field(default_factory=jsonlogger.JsonFormatter)
    level: int = Field(default=logging.INFO)

    class Config:
        arbitrary_types_allowed = True
```

<br>

#### 3.3.5.4 `logger.py` - `BigqueryHandler` 클래스

- `BigqueryHandler`는 `StreamHandler`를 상속받아 사용
- `emit` Method
  - `BigqueryLogSchema` 로그 형태에 맞게 파싱해서 데이터 가공
  - 데이터 전처리하는 코드
- `bigquery_client.insert_rows_json` Method를 사용해 데이터를 실시간으로 저장

```python
class BigqueryHandler(StreamHandler):
    def __init__(self, config: BigqueryHandlerConfig) -> None:
        super().__init__()
        self.config = config
        self.bigquery_client = bigquery.Client(credentials=self.config.credentials)
        self.setLevel(config.level)
        self.setFormatter(fmt=self.config.formatter)

    def emit(self, record: LogRecord) -> None:
        message = self.format(record)
        json_message = json.loads(message)
        log_input = BigqueryLogSchema(
            level=json_message["levelname"],
            message=json_message["message"],
            created_at=datetime.fromtimestamp(
                json_message["created"], tz=pytz.timezone("Asia/Seoul")
            ),
        )
        errors = self.bigquery_client.insert_rows_json(
            self.config.table, [json.loads(log_input.json())]
        )
        if errors:
            print(errors)  # 에러가 발생해도 Logging이 정상적으로 동작하게 하기 위해, 별도의 에러 핸들링을 추가하지 않습니다
```

<br>

#### 3.3.5.5 `logger.py` - main

- 위에서 만든 `get_ml_logger()` 함수를 이용해서, 로그를 각각 파일, console, 그리고 빅쿼리에 저장
- 예시 코드는 hello world를 10번 출력하도록 설정

```python
if __name__ == "__main__":
    from pathlib import Path

    here = Path(__file__)
    config_yaml_path = os.path.join(here.parent, "config.yaml")

    logger = get_ml_logger(
        config_path=config_yaml_path,
        credential_json_path="서비스 계정 JSON 파일 경로",  # FIXME
        table_ref="빅쿼리 테이블 주소",  # FIXME: e.g., boostcamp-ai-tech-serving.online_serving_logs.mask_classification
    )
    for _ in range(10):
        logger.info("hello world")
```

<br>

### 3.3.6 실행 결과

콘솔에서 로그가 정상적으로 나오는 걸 확인

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zHy7ICGUbgdi6b8PFVB5rW_4qsHOHy2F' width=900/>

<br>

파일에도 정상적으로 로그가 저장됨

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zIPhtimVUyM0yRZeRhTk-4csIpd1_lEQ' width=900/>

<br>

BigQuery Table에도 적재된 데이터를 확인할 수 있음

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zM4KhYDmInQ4KdKLkz1gT-rGBalAfGa7' width=600/>

<br>

## 3.4 Special Mission

1. 앞서 만들었던 프로토타입에 실시간 데이터 적재 파이프라인 추가하기 (Input 포함)
2. 만약 이미지 데이터라면 이미지 데이터는 Cloud Stoage에 저장하는 과정 추가하기
3. BigQuery 학습하기
  - 참고 자료 : https://zzsza.github.io/bigquery/